
# Assignment 1: Boolean Model, TF-IDF, and Data Retrieval vs. Information Retrieval Conceptual Questions

**Student names**: Subaial Awan & Las Alei<br>
**Group number**: 79 <br>
**Date**: _Submission Date_

## Important notes
Please carefully read the following notes and consider them for the assignment delivery. Submissions that do not fulfill these requirements will not be assessed and should be submitted again.
1. You may work in groups of maximum 2 students.
2. The assignment must be delivered in ipynb format.
3. The assignment must be typed. Handwritten assignments are not accepted.

**Due date**: 14.09.2025 23:59

In this assignment, you will:
- Implement a Boolean retrieval model
- Compute TF-IDF vectors for documents
- Run retrieval on queries
- Answer conceptual questions 

---
## Dataset

You will use the **Cranfield** dataset, provided in this file:

- `cran.all.1400`: The document collection (1400 documents)

**The code to parse the file is ready — just update the cran file path to match your own file location. Use the docs variable in your code for the parsed file**

### Load and parse documents (provided)

Run the cell to parse the Cranfield documents. Update the path so it points to your `cran.all.1400` file.


In [6]:

# Read 'cran.all.1400' and parse the documents into a suitable data structure

CRAN_PATH = r"/Users/subaialawan/Downloads/Assignment 1 3/cran.all.1400"  # <-- change this!

def parse_cranfield(path):
    docs = {}
    current_id = None
    current_field = None
    buffers = {"T": [], "A": [], "B": [], "W": []}
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.rstrip("\n")
            if line.startswith(".I "):
                if current_id is not None:
                    docs[current_id] = {
                        "id": current_id,
                        "title": " ".join(buffers["T"]).strip(),
                        "abstract": " ".join(buffers["W"]).strip()
                    }
                current_id = int(line.split()[1])
                buffers = {k: [] for k in buffers}
                current_field = None
            elif line.startswith("."):
                tag = line[1:].strip()
                current_field = tag if tag in buffers else None
            else:
                if current_field is not None:
                    buffers[current_field].append(line)
    if current_id is not None:
        docs[current_id] = {
            "id": current_id,
            "title": " ".join(buffers["T"]).strip(),
            "abstract": " ".join(buffers["W"]).strip()
        }
    print(f"Parsed {len(docs)} documents.")
    return docs

docs = parse_cranfield(CRAN_PATH)



Parsed 1400 documents.


## 1.1 – Boolean Retrieval Model

### 1.1.1 Tokenize documents

Implement tokenization using the given list of stopwords. Create a list of normalized terms per document (e.g., lowercase, remove punctuation/digits; drop stopwords). Store the token lists to use in later steps.

In [81]:
# TODO: Implement tokenization using the given list of stopwords, create list of terms per document
import re
import string

STOPWORDS = set("""a about above after again against all am an and any are aren't as at be because been
before being below between both but by can't cannot could couldn't did didn't do does doesn't doing don't down
during each few for from further had hadn't has hasn't have haven't having he he'd he'll he's her here here's hers
herself him himself his how how's i i'd i'll i'm i've if in into is isn't it it's its itself let's me more most
mustn't my myself no nor not of off on once only or other ought our ours ourselves out over own same shan't she
she'd she'll she's should shouldn't so some such than that that's the their theirs them themselves then there there's
these they they'd they'll they're they've this those through to too under until up very was wasn't we we'd we'll we're
we've were weren't what what's when when's where where's which while who who's whom why why's with won't would wouldn't
you you'd you'll you're you've your yours yourself yourselves""".split())


def tokenization(docs):
  
    result = []
    
    for doc in docs.values():
        text = doc["title"] + " " + doc["abstract"]
        
        text = text.lower()
        
        translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
        text = text.translate(translator)
        
        tokens = text.split()
        
        filtered_tokens = [token for token in tokens if token not in STOPWORDS and token.strip()]
        
        result.append(filtered_tokens)
    
    return result

tokenized_documents = tokenization(docs) 


### Build vocabulary

Create a set (or list) of unique terms from all tokenized documents. Report the number of unique terms.


In [82]:
# TODO: Create a set or list of unique terms

# Report: 
# - Number of unique terms

# Your code here

def build_vocabulary(tokenized_documents):
   
    vocabulary = set()
    
    for document_tokens in tokenized_documents:
        vocabulary.update(document_tokens)
        
    
    return vocabulary

vocabulary = build_vocabulary(tokenized_documents)  
print(f"Number of unique terms: {len(vocabulary)}")  


Number of unique terms: 7361


### Build inverted index

For each term, store the list (or set) of document IDs where the term appears.


In [83]:

# TODO: For each term, store list of document IDs where the term appears
# Your code here
def build_inverted_index(tokenized_documents):
  
    inverted_index = {}  
    
    for doc_id, document_tokens in enumerate(tokenized_documents):
        for term in document_tokens:
            if term not in inverted_index:
                inverted_index[term] = set()
            
            inverted_index[term].add(doc_id)
    
    return inverted_index

inverted_index = build_inverted_index(tokenized_documents)


### Retrieve documents for a Boolean query (AND/OR)

Create a function to retrieve documents for a Boolean query (AND/OR) with query terms.  


In [84]:
# TODO: Create a function for retrieving documents for a Boolean query (AND/OR) with query terms

def boolean_retrieve(query: str):
   
    query = query.lower().strip()
    
    if not query:
        return []
    
    tokens = query.split()
    
    filtered_tokens = [token for token in tokens if token in ['and', 'or'] or token not in STOPWORDS and token.strip()]
    
    if not filtered_tokens:
        return []
    
    or_groups = []
    current_group = []
    
    for token in filtered_tokens:
        if token == 'or':
            if current_group:
                or_groups.append(current_group)
            current_group = []
        else:
            current_group.append(token)
    
    if current_group:
        or_groups.append(current_group)
    
    final_results = set()
    
    for group in or_groups:
        terms = [token for token in group if token != 'and']
        
        if not terms:
            continue
        
        group_result = inverted_index.get(terms[0], set()).copy()
        
        for term in terms[1:]:
            term_docs = inverted_index.get(term, set())
            group_result &= term_docs  
        
        final_results |= group_result
    
    return sorted(final_results)



In [85]:
# Do not change this code
boolean_queries = [
  "gas AND pressure",
  "structural AND aeroelastic AND flight AND high AND speed OR aircraft",
  "heat AND conduction AND composite AND slabs",
  "boundary AND layer AND control",
  "compressible AND flow AND nozzle",
  "combustion AND chamber AND injection",
  "laminar AND turbulent AND transition",
  "fatigue AND crack AND growth",
  "wing AND tip AND vortices",
  "propulsion AND efficiency"
]

In [86]:
# Run Boolean queries in batch, using the function you created
def run_batch_boolean(queries):
    results = {}
    for i, q in enumerate(queries, 1):
        res = boolean_retrieve(q)
        results[f"Query {i}"] = res
    return results

boolean_results = run_batch_boolean(boolean_queries)
for qid, res in boolean_results.items():
    print(qid, "=>", res[:5])


Query 1 => [26, 48, 84, 100, 109]
Query 2 => [11, 13, 28, 46, 50]
Query 3 => [4, 398]
Query 4 => [0, 60, 243, 264, 341]
Query 5 => [117, 130]
Query 6 => []
Query 7 => [6, 8, 79, 88, 95]
Query 8 => []
Query 9 => [674]
Query 10 => [967]


## Part 1.2 – TF-IDF Indexing


$tf_{i,j} = \text{Raw Frequency}$

$idf_t = \log\left(\frac{N}{df_t}\right)$

### Build document–term matrix (TF and IDF weights)

Compute tf and idf using the formulas above and store the weights in a document–term matrix (rows = documents, columns = terms).



In [92]:
# TODO: Calculate the weights for the documents and the terms using tf and idf weighting. Put these values into a document–term matrix (rows = documents, columns = terms).
import math
import numpy as np
from collections import defaultdict

def calculate_tf_idf_matrix():
    
    terms_list = sorted(list(vocabulary))  
    num_docs = len(tokenized_documents)  
    num_terms = len(terms_list)  
    
    term_to_idx = {term: idx for idx, term in enumerate(terms_list)}
    
    tf_idf_matrix = np.zeros((num_docs, num_terms))
    
    for doc_idx, doc_tokens in enumerate(tokenized_documents):
        term_counts = defaultdict(int)
        for token in doc_tokens:
            term_counts[token] += 1
        
        for term, count in term_counts.items():
            if term in term_to_idx:
                term_idx = term_to_idx[term]
                
                tf = count
                
                df_t = len(inverted_index.get(term, []))  
                idf = math.log(num_docs / (df_t + 1))  
                
                tf_idf = tf * idf
                tf_idf_matrix[doc_idx, term_idx] = tf_idf
    
    return tf_idf_matrix, terms_list

### Build TF–IDF document vectors

From the matrix, build a TF–IDF vector for each document (consider normalization if needed for cosine similarity).


In [88]:

# TODO: Build TF–IDF document vectors from the document–term matrix
# Your code here
import numpy as np


def build_document_vectors(tf_idf_matrix, normalize=True):
    document_vectors = tf_idf_matrix.copy()
    
    if normalize:
        norms = np.linalg.norm(document_vectors, axis=1, keepdims=True)
        norms = np.where(norms == 0, 1, norms)
        document_vectors = document_vectors / norms
    
    return document_vectors

### Implement cosine similarity

Implement a function to compute cosine similarity scores between a (tokenized) query and all documents.


In [89]:
# TODO: Create a function for calculating the similarity score of all the documents by their relevance to query terms

tf_idf_matrix, terms_list = calculate_tf_idf_matrix()
document_vectors = build_document_vectors(tf_idf_matrix, normalize=True)


def tfidf_retrieve(query: str, k=None):
    query_lower = query.lower()
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    query_clean = query_lower.translate(translator)
    query_tokens = [token for token in query_clean.split() 
                   if token not in STOPWORDS and token.strip()]
    
    if not query_tokens:
        return []
    
    term_to_idx = {term: idx for idx, term in enumerate(terms_list)}
    num_terms = len(terms_list)
    num_docs = len(tokenized_documents)
    
    query_vector = np.zeros(num_terms)
    query_term_counts = Counter(query_tokens)
    
    for term, tf in query_term_counts.items():
        if term in term_to_idx:
            term_idx = term_to_idx[term]
            
            if term in inverted_index:
                df_t = len(inverted_index[term])
                idf = math.log(num_docs / df_t)
                
                query_vector[term_idx] = tf * idf
    
    query_norm = np.linalg.norm(query_vector)
    if query_norm > 0:
        query_vector = query_vector / query_norm
    else:
        return []  
    
    similarity_scores = np.dot(document_vectors, query_vector)
    
    ranked_indices = np.argsort(similarity_scores)[::-1]
    
    ranked_docs = []
    for idx in ranked_indices:
        if similarity_scores[idx] > 0:
            ranked_docs.append(int(idx))
        else:
            break  
    
    if k is not None:
        ranked_docs = ranked_docs[:k]
    
    return ranked_docs

In [90]:
# Do not change this code
tfidf_queries = [
  "gas pressure",
  "structural aeroelastic flight high speed aircraft",
  "heat conduction composite slabs",
  "boundary layer control",
  "compressible flow nozzle",
  "combustion chamber injection",
  "laminar turbulent transition",
  "fatigue crack growth",
  "wing tip vortices",
  "propulsion efficiency"
]

In [91]:
# Run TF-IDF queries in batch (print top-5 results for each), using the function you created
def run_batch_tfidf(queries):
    results = {}
    for i, q in enumerate(queries, 1):
        res = tfidf_retrieve(q)
        results[f"Q{i}"] = res
    return results

tfidf_results = run_batch_tfidf(tfidf_queries)

for qid, res in tfidf_results.items():
    print(qid, "=>", res[:5])


Q1 => [168, 1285, 166, 184, 165]
Q2 => [11, 50, 745, 874, 883]
Q3 => [398, 143, 484, 4, 180]
Q4 => [367, 637, 450, 1162, 1348]
Q5 => [388, 1186, 117, 171, 172]
Q6 => [973, 627, 396, 307, 634]
Q7 => [417, 1263, 314, 271, 1277]
Q8 => [767, 725, 1195, 882, 883]
Q9 => [1283, 432, 1270, 674, 287]
Q10 => [967, 1327, 1379, 1091, 591]



## Part 1.3 – Conceptual Questions

Answer the following questions:

**1. What is the difference between data retrieval and information retrieval?**
Data retrieval deals with structured data and retrieves data based specific queries, where information retrieval deals with unstructured data and retrieves relevant information from a large set of documents.

**For the following scenarios, which approach would be suitable data retrieval or information retrieval? Explain your reasoning.** <br>

1.a A clerk in pharmacy uses the following query: Medicine_name = Ibuprofen_400mg

Since the query is looking for spesific structured data (Medicine_name = Ibuprofen_400mg), data retrieval would be the most suitable approach.

1.b A clerk in pharmacy uses the following query: An anti-biotic medicine 

For this scenario, information retrieval would be more suitable because of "An anti-biotic medicine" is not something spesific but rather a broad thing. This is more a category, which the system would return a list of relevant medicines that match the description.

1.c Searching for the schedule of a flight using the following query: Flight_ID = ZEFV2

Data retrieval. Its looking for exact structured data Flight_ID = ZEFV2.

1.d Searching an E-commerce website using the following query to find an specific shoe: Brooks Ghost 15

Here information retrieval would be more suitable. "Brooks Ghost 15" is a specific search, but the system would have to search through a large collection of unstructured and semi-structured data like images, descriptions etc.

1.e Searching the same E-commerce website using the following query: Nice running shoes

Information retrieval. Nice running shoes is general item and not a specific one.
